In [1]:
import numpy as np
import pandas as pd
import math as m

In [2]:
from sklearn import datasets
iris = datasets.load_iris()

In [3]:
X = pd.DataFrame(iris.data)
Y = pd.DataFrame(iris.target)

In [4]:
X.columns = ['sl','sw','pl','pw']

In [5]:
X.head()

,sl,sw,pl,pw
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
Y.columns = ['flower_type']

In [7]:
Y.head()

,flower_type
0,0
1,0
2,0
3,0
4,0


In [8]:
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [9]:
#Convert all columns to labelled data
X['sl_labeled'] = toLabel(X, 'sl')
X['sw_labeled'] = toLabel(X, 'sw')
X['pl_labeled'] = toLabel(X, 'pl')
X['pw_labeled'] = toLabel(X, 'pw')

In [10]:
# delete the extra columns after labelling
del X['sl']
del X['sw']
del X['pl']
del X['pw']

In [11]:
X.head()

,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a


In [12]:
# splitting the data into testing and training part
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 0)

In [13]:
# entropy of a particular node
def entropy(X,Y):
    count_setosa = 0
    count_versicolor = 0
    count_verginica = 0
    for i in Y:
        if i == 0:
            count_setosa += 1
        elif i == 1:
            count_versicolor += 1
        else:
            count_verginica += 1
    
    total_sum = count_setosa + count_versicolor + count_verginica
    
    entropy_sum_setosa = 0
    entropy_sum_versicolor = 0
    entropy_sum_verginica = 0
    
    if total_sum != 0:
        if count_setosa > 0:
            entropy_sum_setosa = -(count_setosa / total_sum) * (m.log(count_setosa / total_sum , 2))

        if count_versicolor > 0:
            entropy_sum_versicolor = -(count_versicolor / total_sum) * (m.log(count_versicolor / total_sum , 2))

        if count_verginica > 0:
            entropy_sum_verginica = -(count_verginica / total_sum) * (m.log(count_verginica / total_sum , 2))
    
    total_entropy_sum = entropy_sum_setosa + entropy_sum_versicolor + entropy_sum_verginica
    return total_entropy_sum

In [14]:
# entropy of a node on splitting it into various other components
def entropy_node_on_split(X,Y,s_f):
    count_a = [0,0,0]
    count_b = [0,0,0]
    count_c = [0,0,0]
    count_d = [0,0,0]
    for i in range(len(s_f)):
        if s_f[i] == 'a':
            if Y[i] == 0:
                count_a[0] += 1
            elif Y[i] == 1:
                count_a[1] += 1
            else:
                count_a[2] += 1
        elif s_f[i] == 'b':
            if Y[i] == 0:
                count_b[0] += 1
            elif Y[i] == 1:
                count_b[1] += 1
            else:
                count_b[2] += 1
        elif s_f[i] == 'c':
            if Y[i] == 0:
                count_c[0] += 1
            elif Y[i] == 1:
                count_c[1] += 1
            else:
                count_c[2] += 1
        else:
            if Y[i] == 0:
                count_d[0] += 1
            elif Y[i] == 1:
                count_d[1] += 1
            else:
                count_d[2] += 1
        
    total_a = sum(count_a)
    total_b = sum(count_b)
    total_c = sum(count_c)
    total_d = sum(count_d)
    grand_total = total_a + total_b + total_c + total_d
    entropy_a = 0
    entropy_b = 0
    entropy_c = 0
    entropy_d = 0
    for i in range(len(count_a)):
        if count_a[i] > 0 and total_a > 0:
            entropy_a += (-(count_a[i] / total_a)) * m.log(count_a[i] / total_a , 2)
    for i in range(len(count_b)):
        if count_b[i] > 0 and total_b > 0:
            entropy_b += (-(count_b[i] / total_b)) * m.log(count_b[i] / total_b , 2)
    for i in range(len(count_c)):
        if count_c[i] > 0 and total_c > 0:
            entropy_c += (-(count_c[i] / total_c)) * m.log(count_c[i] / total_c , 2)
    for i in range(len(count_d)):
        if count_d[i] > 0 and total_d > 0:
            entropy_d += (-(count_d[i] / total_d)) * m.log(count_d[i] / total_d , 2)
    
    # weighted average sum of all the entropies
    weighted_average = ((total_a / grand_total) * entropy_a) + ((total_b / grand_total) * entropy_b) + ((total_c / grand_total) * entropy_c) + ((total_d / grand_total) * entropy_d)
    return weighted_average

In [15]:
def split_info(X,Y,s_f):
    count_a = 0
    count_b = 0
    count_c = 0
    count_d = 0
    for i in range(len(s_f)):
        if s_f[i] == 'a':
            count_a += 1
        elif s_f[i] == 'b':
            count_b += 1
        elif s_f[i] == 'c':
            count_c += 1
        else:
            count_d += 1
    
    total_count = 0
    total_count = count_a + count_b + count_c + count_d
    A,B,C,D = 0,0,0,0
    if count_a != 0:
        A = (-((count_a) / total_count) * m.log(count_a / total_count , 2))
    
    if count_b != 0:
        B = (-((count_b) / total_count) * m.log(count_b / total_count , 2))
    
    if count_c != 0:
        C = (-((count_c) / total_count) * m.log(count_c / total_count , 2))
        
    if count_d != 0:
        D = (-((count_d) / total_count) * m.log(count_d / total_count , 2))
    
    total_split_info = A + B + C + D
    return total_split_info

In [16]:
def info_gain(X,Y,s_f):
    return entropy(X,Y) - entropy_node_on_split(X,Y,s_f)

In [17]:
def gain_ratio_func(X,Y,s_f):
    return info_gain(X,Y,s_f) / split_info(X,Y,s_f)

In [18]:
# function for printing the leaf node
def print_leaf(X,Y,level,features):
    count = [0,0,0]
    for i in Y:
        if i == 0:
            count[0] += 1
        elif i == 1:
            count[1] += 1
        else:
            count[2] += 1
    print("Level",level)
    print("Count of setosa :",count[0])
    print("Count of versicolor:",count[1])
    print("Count of verginica:",count[2])
    print("Current entropy is = 0.0")
    print("Reached Leaf Node")
    print()

In [19]:
# function for printing the normal node
def print_node(X,Y,s_f,level,entropy,gain_ratio,features,index):
    count = [0,0,0]
    for i in Y:
        if i == 0:
            count[0] += 1
        elif i == 1:
            count[1] += 1
        else:
            count[2] += 1
    
    print("Level",level)
    print("Count of setosa :",count[0])
    print("Count of versicolor:",count[1])
    print("Count of verginica:",count[2])
    print("Current Entropy is =",entropy)
    print("Splitting on feature",s_f,end=" ")
    print("with gain ratio =",gain_ratio)
    
    level += 1
    del features[index]
    return level,features

In [20]:
def build_tree_helper(X,Y,features,level):
    # base case
    if(len(features) == 0 or len(set(Y)) == 1):
        # reached leaf node
        print_leaf(X,Y,level,features)
        return
    
    # lets create a list for the gain ratios for all the features at this level
    gain_ratio = []
    for f in features:
        gain_ratio.append(gain_ratio_func(X,Y,X[f]))
    
    # find the maximum gain ratio
    max_gain_ratio = max(gain_ratio)
    
    # index of that gain ratio
    idx_max_gain = gain_ratio.index(max(gain_ratio))
    selected_feature = features[idx_max_gain]
    
    # find entropy of the selected feature
    entropy_selected_feature = entropy(X,Y)
    
    temp_features = features
    # printing the tree
    new_level,temp_features = print_node(X,Y,selected_feature,level,entropy_selected_feature,max_gain_ratio,temp_features,idx_max_gain)
    print()
    
    level = new_level
    
    # physically split the data upon the selected feature
    #calling for split on a
    new_X_a = X.loc[X[selected_feature] == 'a']
    new_Y_a = pd.DataFrame(Y[list(X[selected_feature] == 'a')])
    new_X_a.reset_index(inplace = True)
    new_Y_a.reset_index(inplace = True)
    del new_X_a['index']
    del new_Y_a['index']
    
    # calling for split on b
    new_X_b = X.loc[X[selected_feature] == 'b']
    new_Y_b = pd.DataFrame(Y[list(X[selected_feature] == 'b')])
    new_X_b.reset_index(inplace = True)
    new_Y_b.reset_index(inplace = True)
    del new_X_b['index']
    del new_Y_b['index']
    
    # calling for split on c
    new_X_c = X.loc[X[selected_feature] == 'c']
    new_Y_c = pd.DataFrame(Y[list(X[selected_feature] == 'c')])
    new_X_c.reset_index(inplace = True)
    new_Y_c.reset_index(inplace = True)
    del new_X_c['index']
    del new_Y_c['index']
    
    # calling for split on d
    new_X_d = X.loc[X[selected_feature] == 'd']
    new_Y_d = pd.DataFrame(Y[list(X[selected_feature] == 'd')])
    new_X_d.reset_index(inplace = True)
    new_Y_d.reset_index(inplace = True)
    del new_X_d['index']
    del new_Y_d['index']
    
    # calling on every split
    if not new_X_a.empty:
        build_tree_helper(new_X_a,new_Y_a['flower_type'],temp_features,level)
        
    if not new_X_b.empty:
        build_tree_helper(new_X_b,new_Y_b['flower_type'],temp_features,level)
    
    if not new_X_c.empty:
        build_tree_helper(new_X_c,new_Y_c['flower_type'],temp_features,level)
    
    if not new_X_d.empty:
        build_tree_helper(new_X_d,new_Y_d['flower_type'],temp_features,level)

In [21]:
def build_tree(X,Y):
    features = list(X.columns)
    return build_tree_helper(X,Y['flower_type'],features,0)

In [22]:
# building the tree
# rebuilding the index of all the data
X_train.reset_index(inplace = True)
Y_train.reset_index(inplace = True)
X_test.reset_index(inplace = True)
Y_test.reset_index(inplace = True)
del X_train['index']
del Y_train['index']
del X_test['index']
del Y_test['index']
build_tree(X_train,Y_train)

Level 0
Count of setosa : 37
Count of versicolor: 34
Count of verginica: 41
Current Entropy is = 1.5807197138422102
Splitting on feature pw_labeled with gain ratio = 0.7026919192529447

Level 1
Count of setosa : 37
Count of versicolor: 0
Count of verginica: 0
Current entropy is = 0.0
Reached Leaf Node

Level 1
Count of setosa : 0
Count of versicolor: 8
Count of verginica: 0
Current entropy is = 0.0
Reached Leaf Node

Level 1
Count of setosa : 0
Count of versicolor: 26
Count of verginica: 11
Current Entropy is = 0.8779620013943912
Splitting on feature pl_labeled with gain ratio = 0.3813740822803121

Level 2
Count of setosa : 0
Count of versicolor: 1
Count of verginica: 0
Current entropy is = 0.0
Reached Leaf Node

Level 2
Count of setosa : 0
Count of versicolor: 25
Count of verginica: 6
Current Entropy is = 0.7088356733321961
Splitting on feature sl_labeled with gain ratio = 0.15312431244209831

Level 3
Count of setosa : 0
Count of versicolor: 0
Count of verginica: 1
Current entropy is 